In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flipkart/flip2.csv


In [ ]:
# TOP OF SCRIPT: Fix tokenizer parallelism and NLTK data
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Install required packages
!pip install transformers datasets scikit-learn nlpaug textattack -q
!pip install nltk==3.8.1 -q  # Specific version to avoid compatibility issues

# Download NLTK data with suppressed messages
import nltk
import logging

# Configure NLTK to be less verbose
nltk_logger = logging.getLogger('nltk')
nltk_logger.setLevel(logging.ERROR)

# Download required NLTK data quietly
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

import pandas as pd
import torch
import numpy as np
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    TrainerCallback,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from torch import nn
import nlpaug.augmenter.word as naw
import random
from tqdm import tqdm

In [36]:
# TOP OF SCRIPT: Fix tokenizer parallelism and NLTK data
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Install required packages
!pip install transformers datasets scikit-learn nlpaug textattack -q
!pip install nltk==3.8.1 -q  # Specific version to avoid compatibility issues

# Download NLTK data with suppressed messages
import nltk
import logging

# Configure NLTK to be less verbose
nltk_logger = logging.getLogger('nltk')
nltk_logger.setLevel(logging.ERROR)

# Download required NLTK data quietly
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

import pandas as pd
import torch
import numpy as np
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    TrainerCallback,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from torch import nn
import nlpaug.augmenter.word as naw
import random
from tqdm import tqdm

# Initialize augmenters once at the start (instead of inside the function)
try:
    # Initialize augmenters outside the function so they're only created once
    aug_synonym = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)
    AUGMENTERS_AVAILABLE = True
except Exception as e:
    print(f"Could not initialize sophisticated augmenters: {str(e)}")
    AUGMENTERS_AVAILABLE = False

def augment_class_samples(df, target_class, target_count, augmenter=None, augment_prob=0.7):
    """
    Augments samples for a specific class to reach target_count
    Args:
        df: DataFrame containing the dataset
        target_class: 'neutral' or 'negative' - class to augment
        target_count: target number of samples for this class
        augmenter: text augmentation model (default: None)
        augment_prob: probability of using augmenter vs simple augmentation
    Returns:
        augmented_df: DataFrame with augmented samples
        report: Dictionary with augmentation statistics
    """
    # Filter samples for the target class
    class_samples = df[df["sentiment"] == target_class]
    current_count = len(class_samples)
    
    # Create report dictionary
    report = {
        "class": target_class,
        "before": current_count,
        "target": target_count,
        "added": 0,
        "after": current_count
    }
    
    # Check if augmentation is needed
    if current_count >= target_count:
        print(f"{target_class.capitalize()} class already has sufficient samples ({current_count}/{target_count})")
        return df, report
        
    num_needed = target_count - current_count
    print(f"\nAugmenting {target_class} samples from {current_count} to {target_count}...")
    report["added"] = num_needed
    
    # Enhanced augmentation techniques
    def enhanced_augment(text):
        words = text.split()
        if len(words) == 0:
            return text
            
        # Random swap
        if len(words) > 1:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
        
        # Random deletion (10-20% of words)
        if len(words) > 3:
            n_deletions = max(1, int(len(words) * random.uniform(0.1, 0.2)))
            for _ in range(n_deletions):
                if len(words) > 2:  # Ensure we don't delete everything
                    del words[random.randint(0, len(words)-1)]
        
        # Random insertion (5-10% of words)
        if len(words) > 2:
            n_insertions = max(1, int(len(words) * random.uniform(0.05, 0.1)))
            for _ in range(n_insertions):
                # Insert a copy of a random word from the text
                word_to_insert = random.choice(words)
                insert_index = random.randint(0, len(words))
                words.insert(insert_index, word_to_insert)
        
        return ' '.join(words)
    
    augmented = []
    pbar = tqdm(total=num_needed, desc=f"Augmenting {target_class} samples")
    
    # Create copies until we reach target count
    while len(augmented) < num_needed:
        for _, row in class_samples.iterrows():
            text = row["text"]
            
            # Apply augmentation
            if augmenter and random.random() < augment_prob:
                try:
                    augmented_text = augmenter.augment(text)
                    if isinstance(augmented_text, list):
                        augmented_text = augmented_text[0]
                except:
                    augmented_text = enhanced_augment(text)
            else:
                augmented_text = enhanced_augment(text)
            
            # Create new row with augmented text
            new_row = row.copy()
            new_row["text"] = augmented_text
            augmented.append(new_row)
            
            pbar.update(1)
            if len(augmented) >= num_needed:
                break
                
    pbar.close()
    
    # Add augmented samples to original dataframe
    augmented_df = pd.DataFrame(augmented)
    result_df = pd.concat([df, augmented_df], ignore_index=True)
    
    # Update report
    report["after"] = len(result_df[result_df["sentiment"] == target_class])
    
    return result_df, report

# =======================
# 2. Load and preprocess data with augmentation
# =======================
csv_path = "/kaggle/input/flipkart/flip2.csv"
df = pd.read_csv(csv_path, header=None)
df.columns = ["ProductName", "Price", "Rating", "Summary", "Review", "sentiment"]  # Changed to lowercase

# Clean and standardize sentiment labels
df["sentiment"] = df["sentiment"].str.strip().str.lower()

# Map to consistent categories
sentiment_mapping = {
    'pos': 'positive',
    'neg': 'negative',
    'neu': 'neutral',
    'positive': 'positive',
    'negative': 'negative',
    'neutral': 'neutral'
}

df["sentiment"] = df["sentiment"].map(sentiment_mapping)
df = df.dropna(subset=["sentiment"])

# Combine Summary + Review for text input
df["text"] = df["Summary"].astype(str) + " " + df["Review"].astype(str)

# Print initial distribution
print("\nClass distribution before augmentation:")
initial_dist = df["sentiment"].value_counts()
print(initial_dist)

# Store augmentation reports
reports = []

# Augment neutral class
df, neutral_report = augment_class_samples(df, "neutral", target_count=1000, augmenter=aug_synonym)
reports.append(neutral_report)

# Print after neutral augmentation
print("\nClass distribution after neutral augmentation:")
print(df["sentiment"].value_counts())

# Augment negative class
df, negative_report = augment_class_samples(df, "negative", target_count=1000, augmenter=aug_synonym)
reports.append(negative_report)

# Print final distribution
print("\nClass distribution after all augmentation:")
final_dist = df["sentiment"].value_counts()
print(final_dist)

# Generate augmentation summary report
print("\n=== Augmentation Summary Report ===")
print(f"{'Class':<10} | {'Before':>6} | {'Target':>6} | {'Added':>6} | {'After':>6} | {'Growth':>7}")
print("-" * 55)
for report in reports:
    growth_pct = (report['after'] - report['before']) / report['before'] * 100
    print(f"{report['class']:<10} | {report['before']:>6} | {report['target']:>6} | "
          f"{report['added']:>6} | {report['after']:>6} | {growth_pct:>6.1f}%")

# Calculate overall growth
total_before = initial_dist.sum()
total_after = final_dist.sum()
growth_pct = (total_after - total_before) / total_before * 100
print("-" * 55)
print(f"{'TOTAL':<10} | {total_before:>6} | {'-':>6} | {total_after - total_before:>6} | "
      f"{total_after:>6} | {growth_pct:>6.1f}%")

# Encode labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["sentiment"])

# Verify class mapping
print("\nLabel Classes:", label_encoder.classes_)

# Create stratified split with augmented data
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df["sentiment"], 
    random_state=42
)

# Convert to DatasetDict
train_dataset = Dataset.from_pandas(train_df[["text", "label"]])
test_dataset = Dataset.from_pandas(test_df[["text", "label"]])
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

# Tokenizer and model setup
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"], 
        padding=False,
        truncation=True, 
        max_length=256
    )

# Tokenize datasets
dataset = dataset.map(
    tokenize, 
    batched=True,
    batch_size=32,
    num_proc=4,
    remove_columns=["text"]
)

# Calculate class weights for loss function
class_counts = train_df["label"].value_counts().sort_index()
total_samples = sum(class_counts)
class_weights = [total_samples / (len(class_counts) * count) for count in class_counts]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# =======================
# 4. Fixed Custom Trainer with Weighted Loss
# =======================
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(self.args.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# =======================
# 5. Load Model
# =======================
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=len(label_encoder.classes_)
)

# =======================
# 6. Data Collator for Dynamic Padding
# =======================
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# =======================
# 7. Enhanced Metrics Function
# =======================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    # Calculate all metrics
    acc = accuracy_score(labels, preds)
    f1_weighted = f1_score(labels, preds, average="weighted")
    f1_macro = f1_score(labels, preds, average="macro")
    
    # Class-wise metrics
    class_report = classification_report(
        labels, preds, 
        labels=list(range(len(label_encoder.classes_))),
        target_names=label_encoder.classes_,
        output_dict=True,
        zero_division=0
    )
    
    return {
        "accuracy": acc,
        "f1_weighted": f1_weighted,
        "f1_macro": f1_macro,
        **{f"{label}_f1": class_report[label]["f1-score"] for label in label_encoder.classes_}
    }

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.3 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.

Class distribution before augmentation:
sentiment
positive    2588
negative     322
neutral       90
Name: count, dtype: int64

Augmenting neutral samples from 90 to 1000...


Augmenting neutral samples:   0%|          | 0/910 [00:00<?, ?it/s][nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already


Class distribution after neutral augmentation:
sentiment
positive    2588
neutral     1000
negative     322
Name: count, dtype: int64

Augmenting negative samples from 322 to 1000...


Augmenting negative samples:   0%|          | 0/678 [00:00<?, ?it/s][nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is alread


Class distribution after all augmentation:
sentiment
positive    2588
negative    1000
neutral     1000
Name: count, dtype: int64

=== Augmentation Summary Report ===
Class      | Before | Target |  Added |  After |  Growth
-------------------------------------------------------
neutral    |     90 |   1000 |    910 |   1000 | 1011.1%
negative   |    322 |   1000 |    678 |   1000 |  210.6%
-------------------------------------------------------
TOTAL      |   3000 |      - |   1588 |   4588 |   52.9%

Label Classes: ['negative' 'neutral' 'positive']


Map (num_proc=4):   0%|          | 0/3670 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/918 [00:00<?, ? examples/s]

The following layers were not sharded: roberta.encoder.layer.*.output.LayerNorm.bias, classifier.out_proj.weight, roberta.encoder.layer.*.attention.output.dense.bias, roberta.embeddings.token_type_embeddings.weight, classifier.dense.bias, roberta.encoder.layer.*.attention.self.value.bias, roberta.encoder.layer.*.attention.self.query.bias, roberta.encoder.layer.*.output.dense.bias, roberta.encoder.layer.*.intermediate.dense.bias, roberta.encoder.layer.*.attention.self.query.weight, roberta.encoder.layer.*.attention.self.key.bias, roberta.encoder.layer.*.attention.self.value.weight, roberta.encoder.layer.*.attention.self.key.weight, roberta.encoder.layer.*.attention.output.LayerNorm.bias, classifier.out_proj.bias, roberta.encoder.layer.*.attention.output.LayerNorm.weight, roberta.embeddings.position_embeddings.weight, roberta.encoder.layer.*.output.LayerNorm.weight, roberta.encoder.layer.*.output.dense.weight, roberta.embeddings.LayerNorm.bias, classifier.dense.weight, roberta.embeddings

In [39]:
# =======================
# 3. Improved Tokenizer & Model with Class Weights
# =======================
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"], 
        padding=False,
        truncation=True, 
        max_length=256
    )

# Create stratified split with augmented data
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df["sentiment"],  # Changed from "Sentiment" to "sentiment"
    random_state=42
)

# Convert to DatasetDict
train_dataset = Dataset.from_pandas(train_df[["text", "label"]])
test_dataset = Dataset.from_pandas(test_df[["text", "label"]])
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

# Tokenize datasets
dataset = dataset.map(
    tokenize, 
    batched=True,
    batch_size=32,
    num_proc=4,
    remove_columns=["text"]
)

# Calculate class weights for loss function
class_counts = train_df["label"].value_counts().sort_index()
total_samples = sum(class_counts)
class_weights = [total_samples / (len(class_counts) * count) for count in class_counts]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# =======================
# 4. Fixed Custom Trainer with Weighted Loss
# =======================
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(self.args.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# =======================
# 5. Load Model
# =======================
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=len(label_encoder.classes_)
)

# =======================
# 6. Data Collator for Dynamic Padding
# =======================
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# =======================
# 7. Enhanced Metrics Function
# =======================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    # Calculate all metrics
    acc = accuracy_score(labels, preds)
    f1_weighted = f1_score(labels, preds, average="weighted")
    f1_macro = f1_score(labels, preds, average="macro")
    
    # Class-wise metrics
    class_report = classification_report(
        labels, preds, 
        labels=list(range(len(label_encoder.classes_))),
        target_names=label_encoder.classes_,
        output_dict=True,
        zero_division=0
    )
    
    return {
        "accuracy": acc,
        "f1_weighted": f1_weighted,
        "f1_macro": f1_macro,
        **{f"{label}_f1": class_report[label]["f1-score"] for label in label_encoder.classes_}
    }

# =======================
# 8. Training Configuration with Early Stopping
# =======================
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,  # Larger batch for evaluation
    num_train_epochs=4,  # Increased epochs
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    disable_tqdm=False,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    load_best_model_at_end=True,
    metric_for_best_model="neutral_f1",  # Focus on neutral class improvement
    greater_is_better=True,
    save_total_limit=2,
    warmup_steps=100
)

# =======================
# 9. Training with Weighted Loss
# =======================
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
print("Starting training...")
trainer.train()
print("Training completed!")

# =======================
# 10. Enhanced Evaluation
# =======================
print("\nEvaluating on test set...")
predictions = trainer.predict(dataset["test"])
preds = np.argmax(predictions.predictions, axis=1)

print("\nDetailed Classification Report:\n", classification_report(
    dataset["test"]["label"],
    preds,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_,
    zero_division=0
))

# Show class distribution for verification
print("\nClass Distribution in Test Set:")
test_labels = [label_encoder.inverse_transform([x])[0] for x in dataset["test"]["label"]]
print(pd.Series(test_labels).value_counts())

# =======================
# 11. Save Model for Deployment
# =======================
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

print("\nModel saved for deployment!")

Map (num_proc=4):   0%|          | 0/3670 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/918 [00:00<?, ? examples/s]

The following layers were not sharded: roberta.encoder.layer.*.output.LayerNorm.bias, classifier.out_proj.weight, roberta.encoder.layer.*.attention.output.dense.bias, roberta.embeddings.token_type_embeddings.weight, classifier.dense.bias, roberta.encoder.layer.*.attention.self.value.bias, roberta.encoder.layer.*.attention.self.query.bias, roberta.encoder.layer.*.output.dense.bias, roberta.encoder.layer.*.intermediate.dense.bias, roberta.encoder.layer.*.attention.self.query.weight, roberta.encoder.layer.*.attention.self.key.bias, roberta.encoder.layer.*.attention.self.value.weight, roberta.encoder.layer.*.attention.self.key.weight, roberta.encoder.layer.*.attention.output.LayerNorm.bias, classifier.out_proj.bias, roberta.encoder.layer.*.attention.output.LayerNorm.weight, roberta.embeddings.position_embeddings.weight, roberta.encoder.layer.*.output.LayerNorm.weight, roberta.encoder.layer.*.output.dense.weight, roberta.embeddings.LayerNorm.bias, classifier.dense.weight, roberta.embeddings

Starting training...


Step,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Negative F1,Neutral F1,Positive F1
100,0.611000,0.775959,0.781046,0.766809,0.696121,0.668305,0.519878,0.900181
200,0.530900,0.704020,0.816993,0.806913,0.745952,0.727689,0.588235,0.921933
300,0.921600,0.639340,0.842048,0.839933,0.797354,0.712575,0.759219,0.920269
400,0.587800,0.609752,0.873638,0.871049,0.836915,0.793566,0.781726,0.935454
500,0.253800,0.407540,0.905229,0.905135,0.881459,0.853012,0.841558,0.949807
600,0.318700,0.456615,0.911765,0.911655,0.890314,0.850123,0.868895,0.951923
700,0.274700,0.380493,0.915033,0.915162,0.891726,0.840796,0.875000,0.959381
800,0.533100,0.428109,0.919390,0.919095,0.901983,0.862745,0.891821,0.951382
900,0.338300,0.352962,0.930283,0.930110,0.915798,0.870466,0.919811,0.957115
1000,0.416400,0.389182,0.932462,0.932147,0.916233,0.865285,0.921241,0.962173


Training completed!

Evaluating on test set...



Detailed Classification Report:
               precision    recall  f1-score   support

    negative       0.92      0.91      0.91       200
     neutral       0.95      0.97      0.96       200
    positive       0.97      0.97      0.97       518

    accuracy                           0.95       918
   macro avg       0.95      0.95      0.95       918
weighted avg       0.95      0.95      0.95       918


Class Distribution in Test Set:
positive    518
neutral     200
negative    200
Name: count, dtype: int64

Model saved for deployment!


In [40]:
from sklearn.metrics import classification_report
import numpy as np

test_preds = trainer.predict(dataset["test"])
test_labels = dataset["test"]["label"]
test_pred_labels = np.argmax(test_preds.predictions, axis=1)

print("Test Set Evaluation:")
print(classification_report(test_labels, test_pred_labels, target_names=label_encoder.classes_))


Test Set Evaluation:
              precision    recall  f1-score   support

    negative       0.92      0.91      0.91       200
     neutral       0.95      0.97      0.96       200
    positive       0.97      0.97      0.97       518

    accuracy                           0.95       918
   macro avg       0.95      0.95      0.95       918
weighted avg       0.95      0.95      0.95       918



In [43]:
# =======================
# 9. Sample Prediction - FIXED DEVICE MISMATCH
# =======================
sample_text = ""
tokens = tokenizer(
    sample_text, 
    return_tensors="pt", 
    padding=True, 
    truncation=True,
    max_length=256
)

# Move tokens to the same device as model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokens = tokens.to(device)
model.to(device)

model.eval()
with torch.no_grad():
    output = model(**tokens)
    
pred_label = torch.argmax(output.logits, dim=1).item()
print("\nSample Text:", sample_text)
print("Predicted Sentiment:", label_encoder.inverse_transform([pred_label])[0])


Sample Text: Hmm it is ohk
Predicted Sentiment: neutral


In [44]:
import shutil

# Compress the folder
shutil.make_archive('sentiment_model', 'zip', 'sentiment_model')



'/kaggle/working/sentiment_model.zip'